In [1]:
import qiskit_qec.utils.pauli_rep as pauli_rep

In [2]:
import copy
import sys

import numpy as np

In [39]:
from qiskit.utils.deprecation import deprecate_function
from qiskit.exceptions import QiskitError

from qiskit_qec.structures.symplectic_matrix import SymplecticMatrixBase
import qiskit_qec.utils.pauli_rep as pauli_rep


class SymplecticMatrixDenseNumPy(SymplecticMatrixBase):
    XZ_ORDER = 0
    ZX_ORDER = 1

    def __init__(self, xz_matrix) -> None:
        """Symplectic Matrix class: None compressed bool matrices using numpy arrays

        Assumption: The assumption is that the X part of the parity check matrix is on
        the left and the Z part is on the right

        Args:
            matrix [SymplecticMatrixBase, ...] : [description]
        """

        # No dimensional check is being done here. Should decided on this
        self._matrix = self.assign(xz_matrix)
        # self._properties = Properties(__class__.__name__)

        super().__init__()

    def __array__(self):
        return self._matrix

    def __repr__(self):
        return np.array2string(self._matrix)

    def __symplectic__(self):
        return self._matrix

    def joinxz(self, x_matrix=None, z_matrix=None):
        """Join matrices into a single symplectic matrix

        Args:
            x_matrix ([type], optional): [description]. Defaults to None.
            z_matrix ([type], optional): [description]. Defaults to None.

            Converts x_matrix and Z_matrix matrices to np.ndarray's and then
            joins matrices together. [x_matrix|z_matrix]
        """
        x_matrix = np.asarray(x_matrix, dtype=bool)
        z_matrix = np.asarray(z_matrix, dtype=bool)
        if x_matrix.shape != z_matrix.shape:
            print("Error: X and Z matrices must have the same shape")
            exit

        return np.hstack(x_matrix, z_matrix)

    def assign(self, matrix):
        """Assign a input symplectic matrix to a Dense NumPy Symplectic matrix
        Includes any necesary checks

        Args:
            matrix ([type], optional): [description].
        """
        # asarray is used so that we do not have to copy the matrix if possible
        # This is needed to allow PauliList to work correctly
        as_matrix = np.asarray(matrix, dtype=bool)
        return as_matrix

    def copy(self):
        """Make a deep copy of the current sympletic matrix"""
        ret = copy.copy(self)
        ret._matrix = self._matrix
        # ret._properties = self._properties.copy()
        return ret

    @property
    def num_qubits(self):
        return self._matrix.shape[1] >> 1

    @property
    def num_paulis(self):
        return self._matrix.shape[0]

    @property
    def x(self):
        """X part of symplectic matrix

        Returns:
            numpy.ndarray: X part of symplectic matrix
        """
        return SymplecticMatrixDenseNumPy(
            xz_matrix=self._matrix[0 : self.num_paulis, 0 : self.num_qubits]
        )

    @property
    def z(self):
        """Z part of symplectic matrix

        Returns:
            numpy.ndarray: Z part of symplectic matrix
        """
        return SymplecticMatrixDenseNumPy(
            xz_matrix=self._matrix[0 : self.num_paulis, self.num_qubits : 2 * self.num_qubits]
        )

    @property
    def num_y(self):
        """Count the number of Y Paulis in each Pauli and return an array of result"""
        return np.sum(np.logical_and(self.x, self.z), axis=1)

    @staticmethod
    def stypes():
        return SymplecticMatrixBase.STYPES

    @staticmethod
    def classnames(key=None):
        if key is not None:
            return SymplecticMatrixBase.CLASSNAMES[key]
        return SymplecticMatrixBase.CLASSNAMES

    @staticmethod
    def logical_and(x1, x2, **kwargs):
        return SymplecticMatrixDenseNumPy(
            xz_matrix=np.logical_and(x1._matrix, x2._matrix, **kwargs)
        )

    @staticmethod
    def logical_xor(x1, x2, **kwargs):
        return SymplecticMatrixDenseNumPy(
            xz_matrix=np.logical_xor(x1._matrix, x2._matrix, **kwargs)
        )

    @staticmethod
    def sum(a, **kwargs):
        return np.sum(a._matrix, **kwargs)

    def __and__(self, other):
        return SymplecticMatrixDenseNumPy(xz_matrix=(self._matrix & other._matrix))

    def __xor__(self, other):
        return SymplecticMatrixDenseNumPy(xz_matrix=(self._matrix ^ other._matrix))

    def __eq__(self, other):
        self._matrix = other._matrix
        self._properties = other._properties

    @staticmethod
    def hstack(tup):
        """Stack arrays in sequence horizontally (column wise)

        This function mimicks numpys hstack for this specific symplectic matrix representation

        Args:
            Tuple of SymplecticMatrixDenseNumPy matrices

        Returns:
            SymplecticMatrixDenseNumPy: The matrix formed by stacking the given matrices
        """

        return SymplecticMatrixDenseNumPy(xz_matrix=np.hstack((item._matrix for item in tup)))

    @staticmethod
    def istack(matrix, size, interleave=False):
        """Vertically stack array of vectors.

        Args:
            SymplecticMatrixDenseNumPy matrice
            size (int)
            interleave (boolean): defulat is False


        matrix = [v_1
                v_2
                ...
                v_k]

        istack(matrix, r, interleave=False) gives r vertically stacked copies of array with no iterleaving

        output = [v_1
                  v_2
                  ...
                  v_k

                  ... r times

                  v_1
                  v_2
                  ...
                  v_k]

        istack(matrix, r, interleave=True) gives r vertically stacked copies of array with with iterleaving

        output = [v_1
                  v_1
                  ... r copies
                  v_1

                  ...

                  v_k
                  v_k
                  ... r copies
                  v_k]

        """
        if size == 1:
            return matrix

        if interleave:
            return np.hstack(size * [matrix._matrix]).reshape(
                (size * len(matrix._matrix),) + matrix._matrix.shape[1:]
            )

        return np.vstack(size * [matrix._matrix]).reshape(
            (size * len(matrix._matrix),) + matrix._matrix.shape[1:]
        )

    @staticmethod
    def count_y(matrix=None, x_matrix=None, z_matrix=None):
        """Count the number of Y Paulis"""
        if matrix is not None:
            return np.sum(np.logical_and(matrix.x, matrix.z), axis=1)
        else:
            return np.sum(np.logical_and(x_matrix, z_matrix), axis=1)

    # This methiod should be deprecated and replaced with _from_matrix
    @staticmethod
    def _from_array(z, x, phase=0, input_format=pauli_rep.DEFAULT_EXTERNAL_PAULI_REP_FORMAT):
        """Convert array data to BasePauli data."""
        # Moved from BasePauli class

        array_z = np.atleast_2d(np.asarray(z, dtype=bool))
        array_x = np.atleast_2d(np.asarray(x, dtype=bool))

        if array_z.ndim != 2:
            raise QiskitError("Invalid Pauli z vector shape.")
        if array_x.ndim != 2:
            raise QiskitError("Invalid Pauli x vector shape.")
        if array_z.shape != array_x.shape:
            raise QiskitError("z and x vectors are different size.")

        # Convert group phase convention to internal ZX-phase conversion.
        # Internal Pauli representation is '-iZX' format
        # External Pauli representation is '-iYZX' format and is assumed as input
        # when referencing is not possible for the phase.
        base_phase = pauli_rep.change_rep(
            phase,
            y_count=SymplecticMatrixDenseNumPy.count_y(x_matrix=array_x, z_matrix=array_z),
            input_format=input_format,
            output_format=pauli_rep.INTERNAL_PAULI_REP_FORMAT,
        )

        return array_z, array_x, base_phase

    @staticmethod
    def _from_matrix(matrix, phase=0, input_format=pauli_rep.DEFAULT_EXTERNAL_PAULI_REP_FORMAT):
        """Convert array data to BasePauli data."""

        matrix = np.atleast_2d(np.asarray(matrix))

        if matrix.shape[1] % 2 != 0:
            raise QiskitError("Invalid matrix shape")

        # Convert group phase convention to internal ZX-phase conversion.
        # Internal Pauli representation is '-iZX' format
        # External Pauli representation is '-iYZX' format and is assumed as input
        # when referencing is not possible for the phase.
        base_phase = pauli_rep.change_rep(
            phase,
            y_count=SymplecticMatrixDenseNumPy.count_y(matrix),
            input_format=input_format,
            output_format=pauli_rep.INTERNAL_PAULI_REP_FORMAT,
        )

        return matrix, base_phase

    @staticmethod
    def _to_matrix(matrix, phase=0, group_phase=False, sparse=False):
        """Return the matrix matrix from symplectic representation.
        The Pauli is defined as :math:`P = (-i)^{phase + z.x} * Z^z.x^x`
        where ``array = [x, z]``.
        Args:
            matrix (SymplecticMatrixDenseNumPy): Symplectic Matrix representation of Pauli
            phase (int): Pauli phase.
            group_phase (bool): Optional. If True use group-phase convention
                                instead of BasePauli ZX-phase convention.
                                (default: False).
            sparse (bool): Optional. Of True return a sparse CSR matrix,
                        otherwise return a dense Numpy array
                        (default: False).
        Returns:
            array: if sparse=False.
            csr_matrix: if sparse=True.
        """
        # Moved from BasePauli
        num_qubits = matrix.z.size
        # Convert to zx_phase

        if group_phase:
            phase += np.sum(x & z)
            phase %= 4
        dim = 2**num_qubits
        twos_array = 1 << np.arange(num_qubits)
        x_indices = np.asarray(x).dot(twos_array)
        z_indices = np.asarray(z).dot(twos_array)
        indptr = np.arange(dim + 1, dtype=np.uint)
        indices = indptr ^ x_indices
        if phase:
            coeff = (-1j) ** phase
        else:
            coeff = 1
        data = np.array([coeff * (-1) ** (bin(i).count("1") % 2) for i in z_indices & indptr])
        if sparse:
            # Return sparse matrix
            from scipy.sparse import csr_matrix

            return csr_matrix((data, indices, indptr), shape=(dim, dim), dtype=complex)
        # Build dense matrix using csr format
        mat = np.zeros((dim, dim), dtype=complex)
        for i in range(dim):
            mat[i][indices[indptr[i] : indptr[i + 1]]] = data[indptr[i] : indptr[i + 1]]
        return mat

    @staticmethod
    def _to_label(z, x, phase, group_phase=False, full_group=True, return_phase=False):
        """Return the label string for a Pauli.
        Args:
            z (array): The symplectic representation z vector.
            x (array): The symplectic representation x vector.
            phase (int): Pauli phase.
            group_phase (bool): Optional. If True use group-phase convention
                                instead of BasePauli ZX-phase convention.
                                (default: False).
            full_group (bool): If True return the Pauli label from the full Pauli group
                including complex coefficient from [1, -1, 1j, -1j]. If
                False return the unsigned Pauli label with coefficient 1
                (default: True).
            return_phase (bool): If True return the adjusted phase for the coefficient
                of the returned Pauli label. This can be used even if
                ``full_group=False``.
        Returns:
            str: the Pauli label from the full Pauli group (if ``full_group=True``) or
                from the unsigned Pauli group (if ``full_group=False``).
            Tuple[str, int]: if ``return_phase=True`` returns a tuple of the Pauli
                            label (from either the full or unsigned Pauli group) and
                            the phase ``q`` for the coefficient :math:`(-i)^(q + x.z)`
                            for the label from the full Pauli group.
        """
        num_qubits = z.size
        coeff_labels = {0: "", 1: "-i", 2: "-", 3: "i"}
        label = ""
        for i in range(num_qubits):
            if not z[num_qubits - 1 - i]:
                if not x[num_qubits - 1 - i]:
                    label += "I"
                else:
                    label += "X"
            elif not x[num_qubits - 1 - i]:
                label += "Z"
            else:
                label += "Y"
                if not group_phase:
                    phase -= 1
        phase %= 4
        if phase and full_group:
            label = coeff_labels[phase] + label
        if return_phase:
            return label, phase
        return label

In [40]:
b = np.array([[1, 1, 1, 1], [1, 0, 1, 0]], dtype=bool)
a = SymplecticMatrixDenseNumPy(b)

In [42]:
print(a.x)

[[ True  True]
 [ True False]]


In [44]:
a.x & a.z

[[ True  True]
 [ True False]]

In [35]:
np.sum(b)

6

In [61]:
a = np.array([[1, 0, 1, 0, 0, 1], [1, 1, 1, 1, 1, 1], [0, 0, 0, 1, 1, 0]])

In [62]:
from qiskit.quantum_info.operators.symplectic.pauli import Pauli
from qiskit.quantum_info.operators.symplectic.pauli_list import PauliList

pl = PauliList(["XXXX", "XYIX", "IIXY"])

In [65]:
%timeit aa=pl[1]

5.53 µs ± 237 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [52]:
%timeit a[1]

151 ns ± 5.94 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [53]:
5150 / 150

34.333333333333336

In [71]:
class newPauliList:
    def __init__(self, matrix):
        self.matrix = np.asarray(matrix)

    def __getitem__(self, slc):
        return newPauliList(self.matrix[slc])

In [203]:
pl = PauliList(["XXXX", "XYIX", "IIXY"])

In [72]:
npl = newPauliList(a)

In [204]:
%timeit bb=pl[1]

5.11 µs ± 272 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [131]:
class Simple:
    def __init__(self, n):
        self.n = n


class SymBase(Simple):
    def __init__(self, matrix):
        if isinstance(matrix, np.ndarray):
            self.matrix = np.asarray(matrix)
        else:
            exit
        super().__init__(matrix.shape[0])


class Sym(SymBase):
    def __init__(self, vector):
        super().__init__(vector)

    def __getitem__(self, slc):
        return self.matrix[slc]


class SymList(SymBase):
    def __init__(self, matrix):
        super().__init__(matrix)

    def __getitem__(self, slc):
        return Sym(self.matrix[slc])

In [137]:
a.shape

(3, 6)

In [132]:
A = SymList(a)

In [134]:
%timeit A[1]

1.21 µs ± 21.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [117]:
%timeit A[1][1]

1.09 µs ± 26.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [92]:
%timeit a[1]

In [136]:
def foo(i):
    for k in range(1, 1000000):
        s = A[i]


import cProfile

cProfile.run("foo(1)")

         6999997 function calls in 2.445 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   999999    0.349    0.000    1.666    0.000 <ipython-input-131-ea575d18ce18>:14(__init__)
   999999    0.116    0.000    0.116    0.000 <ipython-input-131-ea575d18ce18>:2(__init__)
   999999    0.454    0.000    2.121    0.000 <ipython-input-131-ea575d18ce18>:22(__getitem__)
   999999    0.778    0.000    1.318    0.000 <ipython-input-131-ea575d18ce18>:6(__init__)
        1    0.324    0.324    2.445    2.445 <ipython-input-136-90f0c229625d>:1(foo)
        1    0.000    0.000    2.445    2.445 <string>:1(<module>)
   999999    0.178    0.000    0.349    0.000 _asarray.py:14(asarray)
        1    0.000    0.000    2.445    2.445 {built-in method builtins.exec}
   999999    0.074    0.000    0.074    0.000 {built-in method builtins.isinstance}
   999999    0.171    0.000    0.171    0.000 {built-in method numpy.array}
        1    0.000

In [94]:
A[1].matrix[0] = 0

In [95]:
A.matrix

array([[1, 0, 1, 0, 0, 1],
       [0, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 0]])

In [104]:
pl = PauliList(["XXX", "XYX", "IXY"])

In [105]:
%timeit pl[1]

5.26 µs ± 365 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [99]:
a

array([[1, 0, 1, 0, 0, 1],
       [0, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 0]])

In [100]:
v = [1, 2, 3, 4, 5]

In [101]:
%timeit v[1]

34.4 ns ± 0.461 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [102]:
%timeit a[1]

143 ns ± 2.91 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [106]:
class LSymBase:
    def __init__(self, matrix):
        self.matrix = matrix


class LSym(LSymBase):
    def __init__(self, vector):
        super().__init__(vector)

    def __getitem__(self, slc):
        return self.matrix[slc]


class LSymList(LSymBase):
    def __init__(self, matrix):
        super().__init__(matrix)

    def __getitem__(self, slc):
        return LSym(self.matrix[slc])

In [107]:
B = LSymList([[1, 0, 1, 1], [0, 0, 0, 1], [0, 1, 1, 1]])

In [109]:
%timeit B[1]

462 ns ± 11.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [112]:
%timeit B[1][1]

539 ns ± 6.56 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [118]:
C = [[1, 0, 1, 1], [0, 0, 0, 1], [0, 1, 1, 1]]

In [119]:
%timeit C[1]

34 ns ± 1.34 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [120]:
%timeit C[1][1]

48.1 ns ± 0.493 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [122]:
%timeit sum(C[1])

107 ns ± 3.88 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [3]:
class Simple:
    def __init__(self, n):
        self.n = n


class SymBase(Simple):
    def __init__(self, matrix):
        self.matrix = np.asarray(matrix)
        super().__init__(matrix.shape[0])


class Sym(SymBase):
    def __init__(self, vector):
        super().__init__(vector)

    def __getitem__(self, slc):
        return self.matrix[slc]

    def __repr__(self):
        return np.array2string(self.matrix)


class SymList(SymBase):
    def __init__(self, matrix):
        super().__init__(matrix)
        self.paulis = self.init_paulis()

    def init_paulis(self):
        paulis = [Sym(self.matrix[slc]) for slc in range(self.matrix.shape[0])]
        # return np.asarray(paulis)
        return paulis

    def __getitem__(self, slc):
        return self.paulis[slc]

    def __repr__(self):
        return np.array2string(self.matrix)

In [13]:
a = np.random.rand(10000, 100000)
b = a.tolist()
T = SymList(a)

In [17]:
%timeit T[1000]

89.3 ns ± 1.43 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [15]:
%timeit b[100]

29.9 ns ± 0.319 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [195]:
T

[[1 0 1 0 0 1]
 [0 1 1 1 1 1]
 [0 0 0 1 1 0]]

In [200]:
T[0].matrix[1] = 1

In [201]:
T[0]

[1 1 1 0 0 1]

In [202]:
T

[[1 1 1 0 0 1]
 [1 1 1 1 1 1]
 [0 0 0 1 1 0]]

In [205]:
K = [[1, 2, 3, 4], [2, 3, 4, 5], [3, 4, 5, 5]]

In [206]:
%timeit K[1]

33.7 ns ± 2.33 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [48]:
class SMatrix:
    def __init__(self, matrix):
        self.matrix = np.asarray(matrix)

    def __getitem__(self, slc):
        return self.matrix[slc]

In [49]:
sm = SMatrix(a)

In [51]:
sm[1]

array([0.13744246, 0.22582637, 0.96835804, ..., 0.09846227, 0.58292506,
       0.28741057])

In [23]:
%timeit a[1]

137 ns ± 1.39 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [34]:
outp(a,1:2)

SyntaxError: invalid syntax (<ipython-input-34-27a9bb9e7877>, line 1)

In [52]:
a = 1
b = 2
c = "a"
d = "b"

In [53]:
%timeit a==b

36.7 ns ± 0.468 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [54]:
%timeit c==b

38.1 ns ± 0.985 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [57]:
class SMatrix:
    def __init__(self):
        self.sum = np.sum

In [58]:
A = tester()

In [60]:
a = np.array([[1, 1, 1, 1, 1], [2, 3, 1, 2, 1]])

In [61]:
np.sum(a)

14

In [62]:
np.cumsum(a)

array([ 1,  2,  3,  4,  5,  7, 10, 11, 13, 14])

In [63]:
A.sum(a)

14

In [64]:
%timeit np.sum(a)

4.26 µs ± 164 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [65]:
%timeit A.sum(a)

4.28 µs ± 130 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [66]:
tester.gh

AttributeError: type object 'tester' has no attribute 'gh'

In [68]:
class Dispatcher(object):
    def __init__(self, caller, name):
        print(f"name={name}, caller={caller}")
        self.name = name
        self.caller = caller

    def __call__(self, *a, **ka):
        print(
            "Call on Dispatcher registered!",
            "Will create method on",
            self.caller.__class__.__name__,
            "now.",
        )
        setattr(self.caller, self.name, self.mock)
        return getattr(self.caller, self.name)(*a, **ka)

    @classmethod
    def mock(cls, *a, **ka):
        return "Some default value for newly created methods."


class MyClass(object):
    def __getattr__(self, attr):
        return Dispatcher(self, attr)

In [69]:
instance = MyClass()

In [70]:
print(instance.new_method)

name=new_method, caller=<__main__.MyClass object at 0x7fc313326b20>


In [245]:
import warnings
import sys
from scipy import sparse

# import sm.numpy
# import sm.sparse
# import sm.bit


class SMatrix:
    def __new__(cls, matrix, stype="numpy"):
        if stype == "numpy":
            return np.asarray(matrix)
        elif stype == "sparse":
            return sparse.csr_matrix(matrix)
        elif stype == "bit":
            return bit.array(matrix)
        else:
            raise TypeError("Unknown stype")

    def get_methods(stype="numpy"):
        module_name = f"sm.{stype}"
        try:
            module = sys.modules[module_name]
        except KeyError:
            # TODO: If not loaded -> load via __import__
            # Next line for testing only
            return sys.modules["numpy"]
            raise KeyError(f"Module {module_name} not loaded.")
        return module


# BaseOperator, AdjointMixin, MultiplyMixin
class Simple:
    def __init__(self, n):
        self.n = n


# PaulisBase
class SymBase(Simple):
    def __init__(self, matrix, stype="numpy"):
        self.matrix = SMatrix(matrix, stype=stype)
        self.sm = SMatrix.get_methods(stype=stype)
        super().__init__(matrix.shape[0])


# Pauli
class Sym(SymBase):
    def __init__(self, vector):
        super().__init__(vector)

    def __getitem__(self, slc):
        return self.matrix[slc]

    def __repr__(self):
        return np.array2string(self.matrix)


# PauliList
class SymList(SymBase):
    def __init__(self, matrix):
        super().__init__(matrix)
        self.paulis = self.init_paulis()

    def init_paulis(self):
        paulis = [Sym(self.matrix[slc]) for slc in range(self.matrix.shape[0])]
        # return np.asarray(paulis)
        return paulis

    def __getitem__(self, slc):
        return self.paulis[slc]

    def __repr__(self):
        return np.array2string(self.matrix)

In [243]:
a = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
b = np.array([[1, 2, 3, 0], [0, 6, 0, 8]])

In [253]:
%timeit np.sum(a)

3.25 µs ± 113 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [246]:
v = SymList(a)

In [252]:
%timeit v.sm.sum(a)

3.29 µs ± 82.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [181]:
sm = SMatrixMethods_numpy
sm.sum(a)

AttributeError: type object 'SMatrixMethods_numpy' has no attribute 'sum'

In [78]:
smatrix.T

array([[1, 5],
       [2, 6],
       [3, 7],
       [4, 8]])

In [106]:
a

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [107]:
h = sparse.csr_matrix(a)

In [108]:
h.shape

(2, 4)

In [100]:
dir(Foo)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'bar']

In [101]:
a

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [105]:
np.logical_and(a, b)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [103]:
np.sum(a)

36

In [109]:
import sys

In [110]:
sys.modules[__name__]

<module '__main__'>

In [124]:
class myclass:
    def __init__(self):
        name = "test"
        getattr(myclass, name)()

    def test():
        print("Done")

In [125]:
d = myclass()

Done


In [147]:
name = "myclass"
g = getattr(sys.modules[__name__], name)()

Done


In [191]:
class tet:
    def __new__(self, index=None):
        if index is None:
            return "1"
        else:
            return index

    @staticmethod
    def methods(stype="numpy"):
        return "a"

In [192]:
y = tet.methods(1)

In [193]:
name = "np"

'a'

In [194]:
name.array([1, 2])

AttributeError: 'str' object has no attribute 'array'

In [216]:
sys.modules["numpy"]

<module 'numpy' from '/Users/dsvandet/opt/anaconda3/envs/qiskit-latest/lib/python3.8/site-packages/numpy/__init__.py'>

In [217]:
np.__name__

'numpy'

In [209]:
np.__name__

'numpy'

In [223]:
globals()["np"].__name__

'numpy'

In [219]:
import time as gyte

In [222]:
globals()["gyte"].__name__

'time'

In [230]:
a

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [231]:
a.sum()

36

In [233]:
np.sum(a)

36

In [234]:
g = np

In [235]:
g.sum(a)

36

In [236]:
type(np)

module

In [241]:
g = sys.modules["nsfdf"]

KeyError: 'nsfdf'

In [240]:
g.sum(a)

36

In [254]:
import scipy.sparse

In [259]:
a

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [260]:
b = scipy.sparse.csr_matrix(a)

In [261]:
b

<2x4 sparse matrix of type '<class 'numpy.int64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [262]:
import numpy as my.test

SyntaxError: invalid syntax (<ipython-input-262-79331cfe0306>, line 1)

In [263]:
from scipy.sparse import sparse

ImportError: cannot import name 'sparse' from 'scipy.sparse' (/Users/dsvandet/opt/anaconda3/envs/qiskit-latest/lib/python3.8/site-packages/scipy/sparse/__init__.py)

In [2]:
import numpy as np


class SMatrix:
    def __new__(cls, matrix, stype="numpy"):
        """Return a matrix of a specific type depending on the value of stype

        Args:
            matrix ([type]): input matrix to be converted to an stype matrix
            stype (str, optional): type of matrix to return. Defaults to 'numpy'.

        Raises:
            TypeError: An error is raised if an unknown stype is provided.

        Returns:
            stype matrix: a matrix of type stype initialized by the input matrix
        """
        if stype == "numpy":
            return np.asarray(matrix)
        elif stype == "sparse":
            return sparse.csr_matrix(matrix)
        else:
            raise TypeError("Unknown stype")

    def get_methods(stype="numpy"):
        """Return the module that contains the api methods for a given stype matrix

        Args:
            stype (str, optional): type of matrix module api to retrun. Defaults to 'numpy'.

        Raises:
            KeyError: A KeyError will be raised if a needed module that cannot be loaded
            is not loaded.

        Returns:
            module : A module that includes the api for the stype matrices
        """
        # Assumes that the needed modules are of a specific name sm.stype
        #
        # numpy -> sm_numpy
        # sparse -> sm_sparse
        #
        # This needs to be done better
        module_name = f"sm_{stype}"
        try:
            module = sys.modules[module_name]
        except KeyError:
            # TODO: If not loaded -> load via __import__
            raise KeyError(f"Module {module_name} not loaded.")
        return module

In [4]:
a = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])

In [5]:
a

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [6]:
w = SMatrix(a)

In [7]:
w

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [8]:
a

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [13]:
c = np.array([1, 1])

In [14]:
a[c]

array([[5, 6, 7, 8],
       [5, 6, 7, 8]])

In [15]:
a

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [16]:
a[[1, 1]]

array([[5, 6, 7, 8],
       [5, 6, 7, 8]])

In [17]:
a[1, 1]

6

In [19]:
c = a[[0, 0, 1]]

In [20]:
c

array([[1, 2, 3, 4],
       [1, 2, 3, 4],
       [5, 6, 7, 8]])

In [21]:
a[1][1] = 10

In [22]:
c

array([[1, 2, 3, 4],
       [1, 2, 3, 4],
       [5, 6, 7, 8]])

In [23]:
a

array([[ 1,  2,  3,  4],
       [ 5, 10,  7,  8]])

In [24]:
a[1]

array([ 5, 10,  7,  8])

In [25]:
np.atleast_2d(a[1])

array([[ 5, 10,  7,  8]])

In [26]:
from qiskit.quantum_info.operators.symplectic.pauli import Pauli
from qiskit.quantum_info.operators.symplectic.pauli_list import PauliList

In [27]:
k = PauliList(["XXI", "IYX", "YYY"])

In [28]:
k

PauliList(['XXI', 'IYX', 'YYY'])

In [32]:
k._x

array([[False,  True,  True],
       [ True,  True, False],
       [ True,  True,  True]])

In [33]:
k[1]

Pauli('IYX')

In [38]:
k[1:2]

PauliList(['IYX'])

In [35]:
k[(1,)]

Pauli('IYX')

In [37]:
k.to_matrix()

[array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]]),
 array([[0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+

In [1]:
from qiskit_qec.operators.pauli_list import PauliList

SyntaxError: unmatched ')' (paulis_base.py, line 506)

In [2]:
a

NameError: name 'a' is not defined

In [4]:
import numpy as np

a = np.array([[1, 2, 2], [3, 4, 5]])

In [5]:
%timeit a[1]

140 ns ± 4.23 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
